In [7]:
import os
os.chdir('/Users/francopiccolo/GitHub/learn-to-rank')

In [8]:
import lightgbm as lgb

In [12]:
train_ds = lgb.Dataset('./data/train_ds.bin')
val_ds = lgb.Dataset('./data/val_ds.bin')


In [10]:
param = {'objective': 'lambdarank',
         'metric': 'ndcg'}

In [13]:
bst = lgb.train(param, train_ds, valid_sets=[val_ds], early_stopping_rounds=5)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.317193 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4041
[LightGBM] [Info] Number of data points in the train set: 4404529, number of used features: 17
[1]	valid_0's ndcg@1: 0.462951	valid_0's ndcg@2: 0.499878	valid_0's ndcg@3: 0.534065	valid_0's ndcg@4: 0.563419	valid_0's ndcg@5: 0.586883
Training until validation scores don't improve for 5 rounds
[2]	valid_0's ndcg@1: 0.472643	valid_0's ndcg@2: 0.508607	valid_0's ndcg@3: 0.543409	valid_0's ndcg@4: 0.572107	valid_0's ndcg@5: 0.595699
[3]	valid_0's ndcg@1: 0.477688	valid_0's ndcg@2: 0.512847	valid_0's ndcg@3: 0.548142	valid_0's ndcg@4: 0.576186	valid_0's ndcg@5: 0.600057
[4]	valid_0's ndcg@1: 0.479593	valid_0's ndcg@2: 0.515041	valid_0's ndcg@3: 0.549837	valid_0's ndcg@4: 0.578867	valid_0's ndcg@5: 0.602777
[5]	valid_0's ndcg@1: 0.48

In [ ]:
bst.save_model('model.txt', num_iteration=bst.best_iteration)
bst = lgb.Booster(model_file='model.txt')

In [14]:
ypred = bst.predict(test_ds) # num_iteration=bst.best_iteration

TypeError: Cannot use Dataset instance for prediction, please use raw data instead